In [1]:
import pandas as pd
import numpy as np

In [2]:
num_players = 5
deck_df = pd.read_csv('deck.csv')
# Initialize empty dictionary to store player hands
player_hands = {i: [] for i in range(num_players)}


In [ ]:
player_hands

In [3]:
def deck_shuffler(deck_df):
    shuffled_df = deck_df.copy()
    shuffled_df = shuffled_df.iloc[np.random.permutation(len(shuffled_df))]
    return shuffled_df

def initial_deal(shuffled_df, player_hands):
    # Deal 2 cards to each player
    for card_num in range(2):  # Deal 2 rounds
        for player in player_hands.keys():  # Deal to each player
            # Get the next card (first row of shuffled deck)
            card = shuffled_df.iloc[player + (card_num * len(player_hands))]
            # Add card to player's hand as [suit, rank] list
            player_hands[player].append([card['suit'], card['rank']])
    
    # Remove dealt cards from the deck (2 cards * number of players)
    remaining_deck = shuffled_df.iloc[2 * len(player_hands):]
    
    return player_hands, remaining_deck


In [4]:
# Shuffle the deck
shuffled_df = deck_shuffler(deck_df)


In [5]:
player_hands, remaining_deck = initial_deal(shuffled_df, player_hands)


In [ ]:
remaining_deck

In [ ]:
player_hands

In [ ]:
shuffled_df